# Chapter 2 - Statistical Language mModeling
통계 언어 모델링

In [ ]:
# << context >>
# 1 단어 빈도계산 (1gram, 2gram, 3gram)
# 2 주어진 텍스트 MLE 개발 (Maximum Likelihood Estimation : 최대우도 측정모델)
# 3 MLE 모델의 스무딩 적용 
# 4 MLE의 back-off 매커니즘 개발
# 5 mix and match 를 억디위한 데이터 보간법 적용
# 6 혼잡도(perplexity)를 활용한 언어모델의 평가
# 7 Metropolis-Hastings (기각표본추출 알고리즘- 몬테카를로 시뮬레이션에 적용) 를 언어모델링에 적용
# 8 Gibbs sampling을 적용

## 1 단어빈도 의 활용이해
Understanding word frequency

In [3]:
# alpino 말뭉치의 내용 확인
import nltk
from nltk.util import ngrams
from nltk.corpus import alpino
print(alpino.words())

['De', 'verzekeringsmaatschappijen', 'verhelen', ...]


In [8]:
# 1gram 으로 alpino 말뭉치 내용 확인하기
unigrams = ngrams(alpino.words(),1)
for i, word in enumerate(unigrams):
    if i % 20000 == 0: print(word)
print(i)

('De',)
('bundel',)
('rietblazerssectie',)
('we',)
('van',)
('de',)
('hij',)
139819


In [9]:
# 4gram 을 통한 alpino 말뭉치 내용확인
quadgrams = ngrams(alpino.words(),4)

for i, word in enumerate(quadgrams):
    if i % 20000 == 0 : print(word)
print(i)

['De', 'verzekeringsmaatschappijen', 'verhelen', ...]
('De', 'verzekeringsmaatschappijen', 'verhelen', 'niet')
('bundel', 'is', 'het', 'te')
('rietblazerssectie', 'in', 'het', 'orkest')
('we', 'hier', 'Hollandse', 'lijnrechters')
('van', 'Otterloo', 'en', 'het')
('de', 'oostkust', 'van', 'Australie')
('hij', 'vindt', 'er', 'een')
139816


In [22]:
# text를 bigram으로 추출
from nltk.corpus import webtext
tokens=[t.lower()    for t in webtext.words('grail.txt')]
print(len(tokens))
print('grail.txt :', tokens[::2000])  # nltk 기본제공 text 예제

from nltk.collocations import BigramCollocationFinder
words = BigramCollocationFinder.from_words(tokens)

from nltk.metrics import BigramAssocMeasures
print('bi-gram   :' ,words.nbest(BigramAssocMeasures.likelihood_ratio, 10))

16967
grail.txt : ['scene', '?', '[', 'm', "'", 'the', '[', 'pointy', 'no']
bi-gram   : [("'", 's'), ('arthur', ':'), ('#', '1'), ("'", 't'), ('villager', '#'), ('#', '2'), (']', '['), ('1', ':'), ('oh', ','), ('black', 'knight')]


In [23]:
from nltk.corpus import webtext
tokens = [ t.lower()    for t in webtext.words('grail.txt')]

from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
set = set(stopwords.words('english'))
stops_filter = lambda w: len(w) < 3 or w in set
words = BigramCollocationFinder.from_words(tokens)
words.apply_word_filter(stops_filter)

from nltk.metrics import BigramAssocMeasures
print(words.nbest(BigramAssocMeasures.likelihood_ratio, 10))

[('black', 'knight'), ('clop', 'clop'), ('head', 'knight'), ('mumble', 'mumble'), ('squeak', 'squeak'), ('saw', 'saw'), ('holy', 'grail'), ('run', 'away'), ('french', 'guard'), ('cartoon', 'character')]
